# Quadrature Mixers, IQ Demodulation, and the Tayloe Detector - part 3
- https://www.youtube.com/watch?v=JuuKF1RFvBM  
- http://www.analogzoo.com/2015/08/quadrature-mixers/  
- https://en.wikipedia.org/wiki/In-phase_and_quadrature_components

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', 'Waves', 'bitbucket', 'github', 'codes'],
         ['..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline

from waves.sound import *

Populating the interactive namespace from numpy and matplotlib


In [5]:
import warnings 

warnings.filterwarnings('ignore')

In [6]:
from ipywidgets import interactive

In [7]:
def show_pair(sig_left, sig_right, frames = 300, *args, **kwargs):
    sig_left.show(color  = 'red'  , show_frames = slice(0, frames), *args, **kwargs)
    sig_right.show(color = 'green', show_frames = slice(0, frames), *args, **kwargs)

## LO

In [8]:
freq_lo = int(100e3)
freq_sampling = freq_lo * 8 * 2

duration_seconds = 1
t = np.linspace(0, duration_seconds, duration_seconds * freq_sampling)

In [9]:
LO_I = Channel(np.cos(PI2 * freq_lo * t), ts = t, framerate = freq_sampling)
LO_Q = Channel(np.sin(PI2 * freq_lo * t), ts = t, framerate = freq_sampling) 

# show_pair(LO_I, LO_Q, 100)
# plt.figure()
# LO_I.make_spectrum().plot(freq_lims = (1, freq_sampling))

## RF

In [10]:
def make_signal(freq, amp_I = 1.0, amp_Q = 1.0, ts = t, framerate = freq_sampling):
    I = Channel(1  * np.cos(PI2 * freq * t) * amp_I, ts = t, framerate = freq_sampling)
    Q = Channel(1j * np.sin(PI2 * freq * t) * amp_Q, ts = t, framerate = freq_sampling) 
    return  Channel(I.ys + Q.ys, ts = t, framerate = freq_sampling) 

## Demodulation

In [11]:
freq_if = int(1e3)
freq_rf = freq_lo + freq_if
figsize  = (3,2)
 
def process(amp_I = 1.0, amp_Q = 1.0):
#     RF_1 = make_signal(freq = freq_rf, amp_I = amp_I, amp_Q = amp_Q)
#     RF_2 = make_signal(freq = freq_rf, amp_I = 1.0  , amp_Q = 1.0)
    
    RF_1 = make_signal(freq = freq_lo + freq_if, amp_I = amp_I, amp_Q = amp_Q)
    RF_2 = make_signal(freq = freq_lo - freq_if, amp_I = 1.0  , amp_Q = 1.0)

    def demodulat(sig):
        I = sig * LO_I
        Q = sig * LO_Q 
        
        I = I.filter(low_pass_cutoff = freq_if * 2)
        Q = Q.filter(low_pass_cutoff = freq_if * 2)  
        
        return I, Q, I + Q 
    
    IF_1_I, IF_1_Q, IF_1 = demodulat(RF_1)
    IF_2_I, IF_2_Q, IF_2 = demodulat(RF_2)  
    
    ## 兩個 channels 的比較
    plt.figure(figsize = figsize)
    plt.title('Channels')
    show_pair(IF_1, IF_2, 3000)

    ### Cosine portions
    plt.figure(figsize = figsize)
    plt.title('Cosine')
    show_pair(IF_1_I, IF_2_I, 3000)

    ### Sine portions
    plt.figure(figsize = figsize)
    plt.title('Sine')
    show_pair(IF_1_Q, IF_2_Q, 3000)    

In [12]:
interactive(process, amp_I = (0, 1, 1/4), amp_Q = (0, 1, 1/4))

interactive(children=(FloatSlider(value=1.0, description='amp_I', max=1.0, step=0.25), FloatSlider(value=1.0, …

## Summary
- 

#### 用 $cosine$ 訊號做 shifter:
- IF 被 shift 到 以 LO頻率 為中心的 正負 IF頻率
- new frequency = (freq_IF - freq_LO) + (freq_IF + freq_LO)，能量均分  

#### 三角函數的乘法:  

$
\begin{align}
 \sin(x) \cos(y) &= \frac{1}{2} [\sin(x+y) + \sin(x-y)] \\ 
 \cos(x) \sin(y) &= \frac{1}{2} [\sin(x+y) - \sin(x-y)] \\ 
 \cos(x) \cos(y) &= \frac{1}{2} [\cos(x-y) + \cos(x+y)] \\ 
 \sin(x) \sin(y) &= \frac{1}{2} [\cos(x-y) - \cos(x+y)] \\  
\end{align}
$  

####  用 cosine 訊號做 shifter:  

$
\begin{align}
 \cos({\omega_1 t}) \cdot e^{j\omega_2 t} &= \cos(\omega_1 t) \cdot (\cos(\omega_2 t) + j\sin(\omega_2 t)) \\ 
 &= \cos(\omega_1 t) \cos(\omega_2 t) + j\cos(\omega_1 t)\sin(\omega_2 t) \\ 
 &= \frac{1}{2} (\cos((\omega_1 - \omega_2) t) + \cos((\omega_1 + \omega_2) t)) + \frac{1}{2}j \cdot(\sin((\omega_1 + \omega_2) t) -\sin((\omega_1 - \omega_2) t))) \\ 
 &= \frac{1}{2} (\cos((\omega_1 - \omega_2) t) -j\sin((\omega_1 - \omega_2) t) + \cos((\omega_1 + \omega_2) t)) + j\sin((\omega_1 + \omega_2) t) ) \\ 
 &= \frac{1}{2} ((\cos((\omega_1 - \omega_2) t) -j\sin((\omega_1 - \omega_2) t)) + (\cos((\omega_1 + \omega_2) t)) + j\sin((\omega_1 + \omega_2) t))) \\ 
 &= \frac{1}{2} (e^{j(\omega_2 - \omega_1) t} + e^{j(\omega_2 + \omega_1) t}) \\ 
 &= \frac{1}{2} e^{j(\omega_2 - \omega_1) t} + \frac{1}{2}e^{j(\omega_2 + \omega_1) t} \\ 
\end{align}
$

